In [ ]:
## Importamos las librerías necesarias

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import statsmodels.api as sm

In [ ]:
## En esta celda definimos los parámetros del modelo.

total_estampas = 500
estampasXsobre = 5
repetidas_sobre = False
numcorridas = 1000
costosobre = 5

In [ ]:
## esta función genera un sobre de estampas

def sobre(n):
    return np.random.choice(range(1,total_estampas+1),n,replace=repetidas_sobre)

In [ ]:
## Esta función revisa las estampas en un sobre y las tacha de la lista de "No la tengo"

def tachar(sobre, lista):
    for i in sobre:
        if i in lista:
            lista.remove(i)
    return lista

In [ ]:
## Esta función retorna la cantidad de estampas repetidas que salen en cada sobre

def repetidas(sobre,lista):
    return 5 - sum([True for i in sobre if i in lista])

In [ ]:
## Inicializamos las listas para almacenar los valores de las corridas

totsobres = []
totrep = []
perful = []

In [ ]:
## Esta rutina simula el proceso de ir comprando sobres, abriendolos y tachando las que no tenemos, contando las repetidas
## registrando el total de sobres hasta llenar el album y registrar el % de lleno en cada paso.

## Esto se simula para un número de corridas (albums) previamente definidos

for i in range(numcorridas):
    no_la_tengo = list(range(1,total_estampas+1))
    sobres = 0
    rep = []
    porcful = []
    faltan = len(no_la_tengo)
    while len(no_la_tengo) > 0:
        sobres += 1
        a = sobre(estampasXsobre)
        repet = repetidas(a, no_la_tengo)
        rep.append(repet)
        no_la_tengo = tachar(a, no_la_tengo)
        faltan = faltan - (5 - repet)
        porcful.append((total_estampas - faltan)/total_estampas)
    totsobres.append(sobres)
    totrep.append(rep)
    perful.append(porcful)

In [ ]:
## Para sacar promedios de las n corridas, debemos hacerlo para las corridas que estuvieron por debajo del número mínimo de
## sobres que tomó llenar los n álbums.

minimo = min(totsobres)
repprom = []
promlleno = []

In [ ]:
##

for i in range(minimo):
    total = 0
    totlleno = 0
    for j in range(numcorridas):
        total = total + totrep[j][i]
        totlleno = totlleno + perful[j][i]
    repprom.append(total/numcorridas)
    print
    promlleno.append(totlleno/numcorridas)

costoalbum = [costosobre*x for x in totsobres]

In [ ]:
## Graficar las repetidas promedio respecto al % de lleno y al número de sobres

f = plt.figure(figsize=(15,15))
ax1 = f.add_subplot(221)
ax2 = f.add_subplot(222)
ax3 = f.add_subplot(223)
ax4 = f.add_subplot(224)
ax1.plot(repprom)
ax1.set_xlabel('Num Sobres')
ax1.set_ylabel('Repetidas Promedio')
ax2.plot(promlleno,repprom)
ax2.set_xlabel('Porcentaje Lleno')
ax2.set_ylabel('Repetidas Promedio')
ax3.hist(totsobres)
ax3.set_xlabel('Num Sobres')
ax4.hist(costoalbum)
ax4.set_xlabel('Costo en Q')

In [ ]:
## Definir la función de saturación de Michaelis Menten para modelar el promedio de repetidas con respecto al número de sobres.

def MichaelisMenten(x,a,b):
    return a*x/(b+x)

In [ ]:
## Realizar el ajuste de los datos a la función Michaelis Menten para obtener un ecuación

init_vals = [1,1]
y = repprom
x = range(1,len(repprom)+1)
best_vals, covar = curve_fit(MichaelisMenten,x,y,p0=init_vals)

In [ ]:
## Los parámetros a y b de la ecuacióin

best_vals

In [ ]:
## Convertir los valores de x real a una serie de Pandas

xreal = pd.Series(x)

In [ ]:
## Obtener los valores ajustados por medio de la función sobre los valores de x real

ajust = MichaelisMenten(xreal, best_vals[0],best_vals[1])

In [ ]:
## Plotear el valor real de repetidas promedio y el valor ajustado por medio del modelo matemático

plt.rcParams["figure.figsize"] = (12,8)
plt.plot(xreal,repprom,color='blue',label='Datos Reales')
plt.plot(xreal,ajust, color='red',label='Datos Ajustados')
plt.xlabel('# de Sobres')
plt.ylabel('Repetidas Promedio')
plt.legend()
plt.show()

In [ ]:
## Realizar un ajuste lineal de las repetidas promedio respecto al % de lleno promedio del album

X = pd.Series(promlleno)
Y = pd.Series(repprom)
X = sm.add_constant(X)

In [ ]:
## Ajustar a la línea recta  Y =  a + bX

modeloLin = sm.OLS(Y, X).fit()

In [ ]:
## Obtener los valores del modelo para X

pred = modeloLin.predict(X)

print_modelo = modeloLin.summary()
print(print_modelo)

In [ ]:
## Graficar la línea teórica y real del numero promedio de estampas respecto al % de lleno del album

plt.rcParams["figure.figsize"] = (12,8)
plt.plot(X[0],Y,color='blue',label='Datos Reales')
plt.plot(X[0],pred, color='red',label='Datos Ajustados')
plt.xlabel('Porcentaje Lleno')
plt.ylabel('Repetidas Promedio')
plt.legend()
plt.show()